# Step 3: Information Extraction with Custom Lambda Demonstration

This notebook demonstrates the **custom prompt generator Lambda feature** for Patterns 2 and 3. It shows how to:

- Configure custom Lambda functions for extraction prompt generation
- Compare default vs custom prompt extraction results
- Inspect Lambda payloads and responses
- Handle errors and monitor performance

**Prerequisites:**
- Completed Step 2 (Classification)
- AWS Lambda permissions to create/invoke functions
- Demo Lambda function deployed (see deployment section below)

**Key Feature:**
The `custom_prompt_lambda_arn` configuration field allows you to inject custom business logic into the extraction process while leveraging the existing IDP infrastructure.

## 1. Setup and Import Libraries

In [18]:
import os
import json
import time
import logging
import boto3
from pathlib import Path
import yaml

# Import IDP libraries
from idp_common.models import Document, Status
from idp_common import extraction

# Configure logging to see Lambda invocation details
logging.basicConfig(level=logging.INFO)
logging.getLogger("idp_common.extraction").setLevel(logging.INFO)
logging.getLogger("idp_common.bedrock.client").setLevel(logging.INFO)

print("Libraries imported successfully")


## 2. Load Previous Step Data

In [19]:
# Load document from previous step
classification_data_dir = Path(".data/step2_classification")

# Load document object from JSON
document_path = classification_data_dir / "document.json"
with open(document_path, "r") as f:
    document = Document.from_json(f.read())

# Load configuration directly from config files
config_dir = Path("config")
CONFIG = {}

# Load each configuration file
config_files = ["extraction.yaml", "classes.yaml"]

for config_file in config_files:
    config_path = config_dir / config_file
    if config_path.exists():
        with open(config_path, "r") as f:
            file_config = yaml.safe_load(f)
            CONFIG.update(file_config)
        print(f"Loaded {config_file}")
    else:
        print(f"Warning: {config_file} not found")

# Load environment info
env_path = classification_data_dir / "environment.json"
with open(env_path, "r") as f:
    env_info = json.load(f)

# Set environment variables
os.environ["AWS_REGION"] = env_info["region"]
os.environ["METRIC_NAMESPACE"] = "IDP-Custom-Lambda-Demo"

print(f"Loaded document: {document.id}")
print(f"Document status: {document.status.value}")
print(f"Number of sections: {len(document.sections) if document.sections else 0}")
print(f"Loaded configuration sections: {list(CONFIG.keys())}")


## 3. Deploy Demo Lambda Function (Optional)

**Note:** Skip this section if you already have a custom Lambda function deployed.

This section demonstrates how to deploy the demo Lambda function included with this notebook.

In [20]:
# Configuration for demo Lambda deployment
DEMO_LAMBDA_CONFIG = {
    "function_name": "GENAIIDP-notebook-demo-extractor",
    "runtime": "python3.13",
    "handler": "GENAIIDP-notebook-demo-extractor.lambda_handler",
    "timeout": 300,
    "memory_size": 512,
}

# Check if Lambda function already exists
lambda_client = boto3.client("lambda")

try:
    response = lambda_client.get_function(
        FunctionName=DEMO_LAMBDA_CONFIG["function_name"]
    )
    DEMO_LAMBDA_ARN = response["Configuration"]["FunctionArn"]
    print(f"✅ Demo Lambda function already exists: {DEMO_LAMBDA_ARN}")
except lambda_client.exceptions.ResourceNotFoundException:
    print(f"⚠️  Demo Lambda function not found: {DEMO_LAMBDA_CONFIG['function_name']}")
    print(
        "\n📋 To deploy the demo Lambda function, see ./examples/demo-lambda/README.md"
    )
    print(
        "\n💡 For this demo, we'll proceed without custom Lambda first, then show how it works"
    )
    DEMO_LAMBDA_ARN = None
except Exception as e:
    print(f"Error checking Lambda function: {e}")
    DEMO_LAMBDA_ARN = None


## 4. Configure Lambda ARN (Set Your Function ARN Here)

In [21]:
# 🔧 CONFIGURATION: Set your custom Lambda ARN here
# Replace with your actual Lambda function ARN for live testing

# Example ARNs (replace with your actual ARN):
# DEMO_LAMBDA_ARN = "arn:aws:lambda:us-east-1:123456789012:function:GENAIIDP-notebook-demo-extractor"
# DEMO_LAMBDA_ARN = "arn:aws:lambda:us-east-1:123456789012:function:GENAIIDP-my-custom-extractor"

if not DEMO_LAMBDA_ARN:
    print("⚠️  No custom Lambda ARN configured")
    print(
        "💡 This demo will show standard extraction, then simulate custom Lambda behavior"
    )
    print("🔧 To test with a real Lambda, set DEMO_LAMBDA_ARN above")
else:
    print(f"✅ Custom Lambda ARN configured: {DEMO_LAMBDA_ARN}")
    print("🚀 This demo will use your custom Lambda function")


## 5. Extraction Comparison: Default vs Custom Lambda

### 5.1 Default Extraction (Without Custom Lambda)

In [22]:
# Create configuration WITHOUT custom Lambda
config_default = CONFIG.copy()
if "custom_prompt_lambda_arn" in config_default.get("extraction", {}):
    del config_default["extraction"]["custom_prompt_lambda_arn"]

print("=== DEFAULT EXTRACTION CONFIGURATION ===")
print(f"Model: {config_default.get('extraction', {}).get('model')}")
print(
    f"Custom Lambda: {config_default.get('extraction', {}).get('custom_prompt_lambda_arn', 'None')}"
)
print("\n📝 System Prompt (first 200 chars):")
print(config_default.get("extraction", {}).get("system_prompt", "")[:200] + "...")

# Create extraction service with default config
extraction_service_default = extraction.ExtractionService(config=config_default)

print("\n✅ Default extraction service initialized")


In [23]:
# Run default extraction on first section
if document.sections:
    first_section = document.sections[0]
    print(f"🔄 Processing section {first_section.section_id} with DEFAULT prompts")
    print(f"Classification: {first_section.classification}")
    print(f"Pages: {first_section.page_ids}")

    # Save original document state
    document_default = Document.from_json(document.to_json())

    # Process with default extraction
    start_time = time.time()
    document_default = extraction_service_default.process_document_section(
        document=document_default, section_id=first_section.section_id
    )
    default_extraction_time = time.time() - start_time

    print(f"✅ Default extraction completed in {default_extraction_time:.2f} seconds")

    # Store results for comparison
    default_section_result = None
    for section in document_default.sections:
        if section.section_id == first_section.section_id:
            default_section_result = section
            break

else:
    print("⚠️ No sections found in document")


### 5.2 Custom Lambda Extraction

In [24]:
if DEMO_LAMBDA_ARN:
    # Create configuration WITH custom Lambda
    config_custom = CONFIG.copy()
    config_custom["extraction"]["custom_prompt_lambda_arn"] = DEMO_LAMBDA_ARN

    print("=== CUSTOM LAMBDA EXTRACTION CONFIGURATION ===")
    print(f"Model: {config_custom.get('extraction', {}).get('model')}")
    print(f"Custom Lambda: {DEMO_LAMBDA_ARN}")
    print(f"Lambda Function Name: {DEMO_LAMBDA_ARN.split(':')[-1]}")

    # Create extraction service with custom Lambda config
    extraction_service_custom = extraction.ExtractionService(config=config_custom)

    print("\n✅ Custom Lambda extraction service initialized")

else:
    print("⚠️ No custom Lambda ARN configured - skipping custom Lambda demonstration")
    print(
        "💡 Set DEMO_LAMBDA_ARN in the previous cell to test with real Lambda function"
    )
    config_custom = None
    extraction_service_custom = None


In [25]:
# Run custom Lambda extraction on first section
if DEMO_LAMBDA_ARN and document.sections:
    first_section = document.sections[0]
    print(
        f"🔄 Processing section {first_section.section_id} with CUSTOM LAMBDA prompts"
    )
    print(f"Classification: {first_section.classification}")
    print(f"Pages: {first_section.page_ids}")

    # Create fresh document copy for custom processing
    document_custom = Document.from_json(document.to_json())

    # Process with custom Lambda extraction
    start_time = time.time()

    try:
        document_custom = extraction_service_custom.process_document_section(
            document=document_custom, section_id=first_section.section_id
        )
        custom_extraction_time = time.time() - start_time

        print(
            f"✅ Custom Lambda extraction completed in {custom_extraction_time:.2f} seconds"
        )

        # Store results for comparison
        custom_section_result = None
        for section in document_custom.sections:
            if section.section_id == first_section.section_id:
                custom_section_result = section
                break

        # Performance comparison
        overhead = custom_extraction_time - default_extraction_time
        print(f"\n📊 Performance Comparison:")
        print(f"   Default: {default_extraction_time:.2f}s")
        print(f"   Custom:  {custom_extraction_time:.2f}s")
        print(
            f"   Lambda Overhead: {overhead:.2f}s ({overhead / default_extraction_time * 100:.1f}% increase)"
        )

    except Exception as e:
        print(f"❌ Custom Lambda extraction failed: {e}")
        print("\n🔍 This demonstrates the fail-fast error handling behavior")
        custom_section_result = None
        custom_extraction_time = None

else:
    print("⚠️ Skipping custom Lambda extraction (no Lambda configured or no sections)")
    document_custom = None
    custom_section_result = None
    custom_extraction_time = None


## 6. Lambda Payload and Response Inspection

This section shows what data is sent to your custom Lambda and what it should return.

In [26]:
# Demonstrate Lambda payload structure
print("=== LAMBDA PAYLOAD STRUCTURE DEMONSTRATION ===")

if document.sections:
    first_section = document.sections[0]

    # This simulates what the ExtractionService sends to your Lambda
    sample_payload = {
        "config": {
            "extraction": CONFIG.get("extraction", {}),
            "classes": CONFIG.get("classes", [])[:1],  # Show first class only
            "# ... (complete config object)": "...",
        },
        "prompt_placeholders": {
            "DOCUMENT_CLASS": first_section.classification,
            "DOCUMENT_TEXT": "[Document text content would be here...]",
            "ATTRIBUTE_NAMES_AND_DESCRIPTIONS": "[Formatted attribute descriptions...]",
        },
        "default_task_prompt_content": [
            {"text": "[Resolved default task prompt with placeholders replaced...]"},
            {"# Note": "Images and cache points would be included here if present"},
        ],
        "serialized_document": {
            "id": document.id,
            "input_bucket": document.input_bucket,
            "input_key": document.input_key,
            "# ... (complete document object)": "...",
        },
    }

    print("📤 LAMBDA INPUT PAYLOAD STRUCTURE:")
    print(json.dumps(sample_payload, indent=2))

    # Show expected Lambda response format
    sample_response = {
        "system_prompt": "Your custom system prompt for this document type...",
        "task_prompt_content": [
            {"text": "Your custom task prompt with business logic applied..."},
            {"# Note": "You can include images and cache points if needed"},
        ],
    }

    print("\n📥 REQUIRED LAMBDA OUTPUT STRUCTURE:")
    print(json.dumps(sample_response, indent=2))

else:
    print("⚠️ No sections available for payload demonstration")


## 7. Results Comparison: Default vs Custom Lambda

Compare the extraction results between default and custom Lambda processing.

In [27]:
# Helper function to load and display extraction results
def load_and_display_extraction_results(section, label, processing_time=None):
    """Load and display extraction results from a section."""
    print(f"\n--- {label} Results ---")

    if (
        not section
        or not hasattr(section, "extraction_result_uri")
        or not section.extraction_result_uri
    ):
        print("❌ No extraction results available")
        return None

    try:
        # Parse S3 URI and load results
        s3_client = boto3.client("s3")
        uri_parts = section.extraction_result_uri.replace("s3://", "").split("/")
        bucket = uri_parts[0]
        key = "/".join(uri_parts[1:])

        response = s3_client.get_object(Bucket=bucket, Key=key)
        content = response["Body"].read().decode("utf-8")
        result_data = json.loads(content)

        print(f"📍 Result URI: {section.extraction_result_uri}")
        if processing_time:
            print(f"⏱️  Processing Time: {processing_time:.2f} seconds")

        # Display metadata
        if "metadata" in result_data:
            metadata = result_data["metadata"]
            print(f"🔧 Parsing Success: {metadata.get('parsing_succeeded', 'Unknown')}")
            print(
                f"⏱️  Extraction Time: {metadata.get('extraction_time_seconds', 'Unknown')} seconds"
            )

        # Display extracted fields (first few for brevity)
        if "inference_result" in result_data:
            inference_result = result_data["inference_result"]
            print(f"\n📋 Extracted Fields ({len(inference_result)} total):")

            for i, (field_name, field_value) in enumerate(inference_result.items()):
                if i < 5:  # Show first 5 fields
                    display_value = (
                        str(field_value)[:100] + "..."
                        if len(str(field_value)) > 100
                        else field_value
                    )
                    print(f"   {field_name}: {display_value}")
                elif i == 5:
                    print(f"   ... and {len(inference_result) - 5} more fields")
                    break

        return result_data

    except Exception as e:
        print(f"❌ Error loading results: {e}")
        return None


# Display default extraction results
if "default_section_result" in locals():
    default_results = load_and_display_extraction_results(
        default_section_result, "DEFAULT EXTRACTION", default_extraction_time
    )
else:
    print("⚠️ No default results to display")
    default_results = None


In [28]:
# Display custom Lambda extraction results
if DEMO_LAMBDA_ARN and "custom_section_result" in locals():
    custom_results = load_and_display_extraction_results(
        custom_section_result, "CUSTOM LAMBDA EXTRACTION", custom_extraction_time
    )
else:
    print("\n⚠️ No custom Lambda results to display")
    custom_results = None


## 8. Error Handling Demonstration

In [31]:
# Demonstrate error handling with invalid Lambda ARN
print("=== ERROR HANDLING DEMONSTRATION ===")

# Create config with invalid Lambda ARN to show error behavior
config_error_demo = CONFIG.copy()
config_error_demo["extraction"]["custom_prompt_lambda_arn"] = (
    "arn:aws:lambda:us-east-1:123456789012:function:GENAIIDP-nonexistent-function"
)

print(
    f"🧪 Testing with invalid Lambda ARN: {config_error_demo['extraction']['custom_prompt_lambda_arn']}"
)
print("📋 Expected behavior: Extraction should fail with clear error message")

# Uncomment the lines below to test error handling (will cause intentional failure)
# extraction_service_error = extraction.ExtractionService(config=config_error_demo)
#
# if document.sections:
#     try:
#         document_error = Document.from_json(document.to_json())
#         document_error = extraction_service_error.process_document_section(
#             document=document_error,
#             section_id=document.sections[0].section_id
#         )
#     except Exception as e:
#         print(f"✅ Expected error occurred: {e}")
#         print("🔍 This demonstrates fail-fast behavior when Lambda fails")

print("\n💡 Uncomment the code above to see actual error handling behavior")
print("🛡️  The system is designed to fail fast with clear error messages")


## 9. CloudWatch Logs Monitoring

Monitor the logs to understand Lambda invocation behavior.

In [32]:
print("=== CLOUDWATCH LOGS MONITORING GUIDE ===")

if DEMO_LAMBDA_ARN:
    lambda_function_name = DEMO_LAMBDA_ARN.split(":")[-1]

    print(f"📊 Monitor these CloudWatch Log Groups:")
    print(f"   🔧 Custom Lambda Logs: /aws/lambda/{lambda_function_name}")
    print(
        f"   📋 IDP Extraction Logs: Look for 'extraction' in your IDP stack log groups"
    )

    print(f"\n🔍 Key log messages to look for:")
    print(f"   ✅ 'Using custom prompt Lambda: {DEMO_LAMBDA_ARN}'")
    print(f"   ✅ 'Custom prompt Lambda invoked successfully'")
    print(f"   ✅ 'Successfully applied custom prompt from Lambda function'")

    print(f"\n🔍 Demo Lambda specific messages:")
    print(f"   📋 '=== DEMO LAMBDA INVOKED ==='")
    print(f"   🧠 'DEMO LOGIC: Applying [document type] customization'")
    print(f"   📊 'DEMO ANALYSIS: [Business logic decisions]'")
    print(f"   ✅ 'DEMO RESULT: Generated specialized [type] prompts'")

    print(f"\n📈 AWS Console Links:")
    region = os.environ.get("AWS_REGION", "us-east-1")
    print(
        f"   Lambda Function: https://console.aws.amazon.com/lambda/home?region={region}#/functions/{lambda_function_name}"
    )
    print(
        f"   Lambda Logs: https://console.aws.amazon.com/cloudwatch/home?region={region}#logsV2:log-groups/log-group/$252Faws$252Flambda$252F{lambda_function_name}"
    )

else:
    print("⚠️ No Lambda ARN configured - no specific logs to monitor")
    print(
        "💡 When you configure a Lambda, monitor its CloudWatch logs for decision-making insights"
    )


## 10. Save Results and Summary

In [33]:
# Create data directory for this step
data_dir = Path(".data/step3_extraction_with_custom_lambda")
data_dir.mkdir(parents=True, exist_ok=True)

# Use the best result (custom if available, otherwise default)
final_document = (
    document_custom if DEMO_LAMBDA_ARN and document_custom else document_default
)

# Save updated document object as JSON
document_path = data_dir / "document.json"
with open(document_path, "w") as f:
    f.write(final_document.to_json())

# Save configuration (with custom Lambda if used)
config_used = config_custom if DEMO_LAMBDA_ARN and config_custom else config_default
config_path = data_dir / "config.json"
with open(config_path, "w") as f:
    json.dump(config_used, f, indent=2)

# Save environment info (pass through)
env_path = data_dir / "environment.json"
with open(env_path, "w") as f:
    json.dump(env_info, f, indent=2)

# Create comprehensive demo summary
demo_summary = {
    "demo_configuration": {
        "custom_lambda_arn": DEMO_LAMBDA_ARN,
        "lambda_configured": DEMO_LAMBDA_ARN is not None,
        "model_used": CONFIG.get("extraction", {}).get("model"),
    },
    "processing_results": {
        "sections_processed": 1 if document.sections else 0,
        "total_sections": len(document.sections) if document.sections else 0,
        "default_processing_time": default_extraction_time
        if "default_extraction_time" in locals()
        else None,
        "custom_processing_time": custom_extraction_time
        if "custom_extraction_time" in locals()
        else None,
        "lambda_overhead_seconds": (custom_extraction_time - default_extraction_time)
        if all(
            v in locals() for v in ["custom_extraction_time", "default_extraction_time"]
        )
        else None,
    },
    "comparison_available": {
        "default_results": default_results is not None,
        "custom_results": custom_results is not None
        if "custom_results" in locals()
        else False,
        "both_available": default_results is not None
        and ("custom_results" in locals() and custom_results is not None),
    },
    "demo_insights": {
        "feature_demonstrated": "Custom prompt generator Lambda integration",
        "key_capabilities": [
            "Document type detection and specialized prompt generation",
            "Content-based analysis for conditional processing",
            "Business logic integration with existing IDP infrastructure",
            "Fail-fast error handling for Lambda failures",
            "Performance monitoring and comparison",
        ],
    },
}

demo_summary_path = data_dir / "demo_summary.json"
with open(demo_summary_path, "w") as f:
    json.dump(demo_summary, f, indent=2)

print(f"💾 Saved demo results to: {data_dir}")
print(f"📋 Document: {document_path}")
print(f"⚙️  Configuration: {config_path}")
print(f"🌍 Environment: {env_path}")
print(f"📊 Demo Summary: {demo_summary_path}")


## 11. Next Steps and Production Considerations

In [34]:
print("=== DEMO COMPLETE: NEXT STEPS ===")

sections_processed = 1 if document.sections else 0
lambda_used = DEMO_LAMBDA_ARN is not None

print(f"\n✅ DEMO RESULTS:")
print(f"   📄 Document processed: {document.id}")
print(f"   📊 Sections processed: {sections_processed}")
print(f"   🔧 Custom Lambda used: {'Yes' if lambda_used else 'No (simulated)'}")
print(f"   📈 Performance overhead: {'Measured' if lambda_used else 'Simulated'}")

print(f"\n🚀 TO IMPLEMENT CUSTOM LAMBDA IN PRODUCTION:")
print(f"   1. 📝 Create your Lambda function with GENAIIDP-* naming")
print(f"   2. 🔐 Deploy with appropriate IAM role and permissions")
print(f"   3. ⚙️  Add 'custom_prompt_lambda_arn' to your extraction config")
print(f"   4. 🧪 Test with your actual documents and use cases")
print(f"   5. 📊 Monitor CloudWatch logs for performance and errors")

print(f"\n📚 ADDITIONAL RESOURCES:")
print(f"   📖 Documentation: examples/custom-prompt-lambda/README.md")
print(
    f"   🔧 Full Example Lambda: examples/custom-prompt-lambda/GENAIIDP-example-custom-prompt-generator.py"
)
print(
    f"   📓 Demo Lambda Code: notebooks/examples/demo-lambda/GENAIIDP-notebook-demo-extractor.py"
)

print(f"\n📌 CONTINUE TO: step4_assessment.ipynb")
print(
    f"   💡 The custom Lambda feature works with all downstream processing (assessment, summarization, etc.)"
)
